## AWS ElasticSearch KNN Tests

In [108]:
from elasticsearch import Elasticsearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

In [157]:
# host = '' # For example, my-test-domain.us-east-1.es.amazonaws.com
# region = '' # e.g. us-west-1

#host = "search-alion-test2-4sb55az5ow7h7koojlem6w7ria.us-east-1.es.amazonaws.com/"
#host = "https://vpc-aliontest2-qjamtzemjlnb2hkttspsj67x3i.us-east-1.es.amazonaws.com"
#host = "https://vpc-aliontest2-qjamtzemjlnb2hkttspsj67x3i.us-east-1.es.amazonaws.com/"
#host = "https://search-aliontest3-suqzkmvjgko3qpaykksy6iz6ci.us-east-1.es.amazonaws.com"
#host = "https://search-aliontest4-jz4uagf2mg56r6xj5wp7kqlmpq.us-east-1.es.amazonaws.com"


"""
notes to future setup
test
host is the vpc endpoint of the ES domain minus the 'https://' and '/' at the start and end


"""


host = "search-aliontest1-vvw7pstcvgijdcmwjl65n2dxpa.us-east-1.es.amazonaws.com"
region = "us-east-1"

In [158]:
service = "es"
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)


In [111]:
### Check can connect to host

#response = client.describe_elasticsearch_domain(
#    DomainName='string'
#)


# host = "search-alion-test-6yrydpasjr7pyfktt3odnhpbxy.us-east-1.es.amazonaws.com"
# host = "https://alion-test.us-east-1.es.amazonaws.com"


print(host)
es = Elasticsearch(
    hosts = [{'host': host, 'port': 443}],
#      http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

print(es.info())

search-aliontest1-vvw7pstcvgijdcmwjl65n2dxpa.us-east-1.es.amazonaws.com
{'name': 'd84ac9305698b88b417cf6867afb5622', 'cluster_name': '003440510767:aliontest1', 'cluster_uuid': '8EmQDY1RTceu05N9ujjtSA', 'version': {'number': '7.4.2', 'build_flavor': 'oss', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2020-05-05T04:47:06.936807Z', 'build_snapshot': False, 'lucene_version': '8.2.0', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


In [149]:
### create knn index as type doc

index_name = "knn"
created = False
# index settings
settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
                "properties": {
                    "my_vector": {
                        "type": "knn_vector",
                        "dimension": 768
                    },
                    "keywords": {
                        "type": "keyword"
                    }
                }
            
        }
    }
try:
    if not es.indices.exists(index_name):
        es.indices.create(index=index_name, body=settings)
        print('Created Index')
    else:
        print("index exists")
except Exception as ex:
    print(str(ex))
    

Created Index


In [150]:
### search the new index to check is created


es.search(index='knn',body=None)



{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [151]:
### add 2 test keywords

import numpy as np

index_name = "knn"
index_property = "keywords"

index_name = "knn"
vector_name = "my_vector"
keyword_name = "testkeyword"

embedding= np.random.rand(768).tolist()


r = es.bulk(
    body=[
    { "index": {"_index":index_name, "_id": 2 }},
    { vector_name: embedding, "price":1,keyword_name: ["test1","test2"]}
    ])

print(r)


# doc ex for bulk
"""
es.bulk(
    body=[
        {"index": {"_index": "test", "_id": "1"}},
        {"field1": "value1"},
        {"delete": {"_index": "test", "_id": "2"}},
        {"create": {"_index": "test", "_id": "3"}},
        {"field1": "value3"},
        {"update": {"_id": "1", "_index": "test"}},
        {"knn_vector": {"keyword": "test"}},
    ],
)
"""


{'took': 24, 'errors': False, 'items': [{'index': {'_index': 'knn', '_type': '_doc', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}]}


'\nes.bulk(\n    body=[\n        {"index": {"_index": "test", "_id": "1"}},\n        {"field1": "value1"},\n        {"delete": {"_index": "test", "_id": "2"}},\n        {"create": {"_index": "test", "_id": "3"}},\n        {"field1": "value3"},\n        {"update": {"_id": "1", "_index": "test"}},\n        {"knn_vector": {"keyword": "test"}},\n    ],\n)\n'

### Bulk Upload

In [152]:
# num_passage = int(1e7) # 10 million
num_passage_per_batch = 20#int(1e2)
num_batch = 100 # half number of batches cause Im on spotty internet
print("Total passages in bulk: {}".format(num_passage_per_batch*num_batch))

# INDEX_NAME = "knn-test"

Total passages in bulk: 2000


In [154]:
import json
import time
import numpy as np
import requests
from tqdm import tqdm

# for testing
import random

# bulk_file = ''

index_name = "knn"
vector_name = "my_vector"
vector_dimension = 768
keyword_name = "keywords"

index_part =     { "index": {"_index":index_name, "_id": 1 }}


keyword_options = ["test","boston","harbor","tour","boat","atlantic","sea","seal","whale","ocean"]

id = 20030000+12700000
time_sp = time.time()
for _iter in range(num_batch):
    print("Current iter num: {}".format(_iter+1))

    bulk_body = []
    
    for _ in tqdm(range(num_passage_per_batch)):
        
        # add index step to body
        index_part  = { "index": {"_index":index_name, "_id": id }}
        bulk_body.append(index_part)

        # value part to be added to body
        value_part = dict()
        
        # add knn vector step to value
        value_part[vector_name] = embedding
        
        # add keyword (1-3) part to value
        # random add keywords for testing
        # keywords is always an array [] of keyword objects even if length 1
        keywords = []
        num_keywords = random.randint(1,3)
        for n in range(num_keywords):
            keywords.append(random.choice(keyword_options))
            
        value_part[keyword_name] = keywords
        # add value to body
        bulk_body.append(value_part)

        # increment/generate values
        embedding= np.random.rand(vector_dimension).tolist()
        id += 1
    
    # the body is now 2 elements, the index point, then a value to add to it
    # { "index": {"_index":index_name, "_id": 2 }},
    # { vector_name: embedding, "price":1,keyword_name: "test2"}
    
    
    es.bulk(bulk_body)#,index=index_name)
    avg_batch_time = round((time.time() - time_sp) / (1 + _iter),4)
    print("Average batch time for batch {} is {}".format(1+_iter,avg_batch_time))


100%|██████████| 20/20 [00:00<00:00, 12803.13it/s]

Current iter num: 1



100%|██████████| 20/20 [00:00<00:00, 7730.01it/s]

Average batch time for batch 1 is 1.3299
Current iter num: 2



100%|██████████| 20/20 [00:00<00:00, 8018.17it/s]

...


100%|██████████| 20/20 [00:00<00:00, 6049.77it/s]

Average batch time for batch 98 is 0.7414
Current iter num: 99



100%|██████████| 20/20 [00:00<00:00, 5587.19it/s]

Average batch time for batch 99 is 0.7372
Current iter num: 100


Average batch time for batch 100 is 0.7332


#### Query based on fuzzy match keyword




In [156]:
### search the to see results are uploaded


max_results = 1000
search_query = "bosto"

query =  {
    "size": max_results,
    "query": {
        "fuzzy" : {
            "keywords" : search_query
        }
    }
}
# `fuzzy` can be changed to `match` for exact match search

r = es.search(index='knn',body=query)
# r is result json

hits = r["hits"]["hits"]

# print all keywords found
for hit in hits:
    keywords = hit["_source"]["keywords"]
    # hit[_source][my_vector] is the vector with those keywords
    print(keywords)

print(len(hits))


['boat', 'boston', 'harbor']
['boston', 'atlantic']
['boston']
['harbor', 'boston']
['boston']
['boston', 'test']
['boston', 'boston']
['seal', 'boston']
['seal', 'boston', 'sea']
['atlantic', 'boston', 'tour']
['boston', 'boston']
['sea', 'boston', 'harbor']
['boston', 'atlantic', 'test']
['boston', 'ocean']
['boston', 'ocean', 'tour']
['boston', 'test']
['boston']
['boat', 'boston', 'test']
['seal', 'whale', 'boston']
['boston', 'boat']
['sea', 'boston', 'boat']
['tour', 'boston']
['boston', 'sea']
['boston']
['ocean', 'atlantic', 'boston']
['boston']
['seal', 'ocean', 'boston']
['boston', 'boat']
['boston']
['boston', 'seal']
['seal', 'harbor', 'boston']
['boston', 'sea', 'harbor']
['sea', 'boston', 'harbor']
['boston', 'sea', 'atlantic']
['boat', 'boat', 'boston']
['harbor', 'boston', 'tour']
['boston', 'harbor', 'seal']
['whale', 'boston', 'harbor']
['ocean', 'boston', 'boston']
['boston', 'boston']
['boston']
['boston', 'harbor', 'whale']
['tour', 'boston', 'harbor']
['sea', 'wha

#### Query based on query embedding

In [139]:
import numpy as np

NUM_OUTPUT = 1000
QUERY_EMBEDDING = np.random.rand(1024).tolist()

es_query = {
  "size": NUM_OUTPUT,
  "query": {
    "knn": {
      "my_vector1": {
        "vector": QUERY_EMBEDDING,
        "k": NUM_OUTPUT
      }
    }
  }
}





es_query = {
  "size": NUM_OUTPUT,
  "query": {
    "knn": {
      "my_vector1": {
        "vector": QUERY_EMBEDDING,
        "k": NUM_OUTPUT
      }
    }
  }
}

In [140]:
import requests

#host = "https://search-alion-test3-cbeplr3jqsbbwzbqbeuryjstu4.us-east-1.es.amazonaws.com/"

endpoint = "knn/_search"

url = "https://"+host+"/" + endpoint
print(url)

https://search-aliontest1-vvw7pstcvgijdcmwjl65n2dxpa.us-east-1.es.amazonaws.com/knn/_search


In [141]:
r = requests.get(url, json=es_query)

print(r.text)

{"took":1,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":0,"relation":"eq"},"max_score":null,"hits":[]}}


In [142]:
r.status_code

200

In [143]:
import ast 
import json

# result = ast.literal_eval(r.text)
result = json.loads(r.text)

In [144]:
result.keys()

dict_keys(['took', 'timed_out', '_shards', 'hits'])

In [145]:
result['hits'].keys()

dict_keys(['total', 'max_score', 'hits'])

In [146]:
# result_id['_id']

In [147]:
len(result['hits']['hits'])

0

In [148]:
from sklearn.metrics.pairwise import cosine_similarity

for _idx in range(10):
    result_id = result['hits']['hits'][_idx]['_id']
    result_embedding = result['hits']['hits'][_idx]['_source']['my_vector1']
    result_score = result['hits']['hits'][_idx]['_score']
    print("AWS ES score for id {} is {}\n".format(result_id,result_score))
    
    cos_score = cosine_similarity([QUERY_EMBEDDING, result_embedding])
    print("Sklearn cosine score for id {} is {}\n".format(result_id,round(cos_score[0][1],4)))
# result['hits']['hits'][1]

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
result['hits']['hits'][_idx]